# Multi-stage Uncertainty ML

In [1]:
pip list

Package                       Version
----------------------------- ------------------------------
absl-py                       1.0.0
aiosignal                     1.3.1
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
asttokens                     2.1.0
astunparse                    1.6.3
attrs                         22.1.0
backcall                      0.2.0
beautifulsoup4                4.11.1
bleach                        5.0.1
blinker                       1.4
cachetools                    5.2.0
certifi                       2022.9.24
cffi                          1.15.1
chardet                       3.0.4
charset-normalizer            2.1.1
clang                         13.0.1
click                         8.0.4
cloudpickle                   2.2.0
cryptography                  2.8
cuda-python                   11.7.0+0.g95a2041.dirty
cudf                          22.8.0a0+304.g6ca81bbc78.dirty
cugraph                       22.8.0a0+132.g2daa31b6.dirty
c

In [ ]:
# Seeds
import numpy as np
import os, os.path
import math 

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CUDNN_DETERMINISTIC'] = 'true'
os.environ['TF_DETERMINISTIC_OPS'] = 'true'

import random as python_random
from imblearn.over_sampling import SMOTE
import time
# Data wrangling
import numpy as np
import pandas as pd  # Not a requirement of giotto-tda, but is compatible with the gtda.mapper module
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.ndimage import rotate

# TDA
import gtda.mapper 
from sklearn.utils import shuffle

# Data viz
from gtda.plotting import plot_point_cloud

# TDA magic
from gtda.mapper import (
    CubicalCover,
    OneDimensionalCover,
    make_mapper_pipeline,
    Projection,
    plot_static_mapper_graph,
    plot_interactive_mapper_graph,
    MapperInteractivePlotter
)

# ML tools
from sklearn import datasets
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# Sklearn_TDA and Stat_mapper 

from sklearn.cluster import AgglomerativeClustering, KMeans
from itertools import product

# Persistent Homology Machine Learning 
from gtda.homology import CubicalPersistence
import tensorflow as tf
from matplotlib import pyplot as plt
from gtda.images import DensityFiltration
import keras_tuner
from gtda.diagrams import Amplitude
from sklearn.pipeline import make_union
from gtda.diagrams import PersistenceEntropy
from gtda.diagrams import NumberOfPoints
from sklearn.model_selection import KFold, StratifiedKFold, ShuffleSplit, StratifiedShuffleSplit
from sklearn import preprocessing
from sklearn.feature_selection import RFE, f_classif, SelectKBest, VarianceThreshold, mutual_info_classif, RFECV
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn import metrics
from sklearn import model_selection
from gtda.diagrams import Scaler
from sklearn.inspection import permutation_importance
 
def reset_seeds(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed) 
    python_random.seed(seed)
    tf.random.set_seed(1)
    
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K

from keras import callbacks
# Keras model things
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers.core import Dense, Flatten, Activation, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers import concatenate, Input, Dropout, BatchNormalization, SpatialDropout2D, Lambda, Conv2DTranspose, Reshape, UpSampling2D
import tensorflow_addons as tfa
from tensorflow.python.saved_model import loader_impl
from tensorflow.python.saved_model import load as tf_load
from tensorflow.python.keras.saving import saving_utils
from tensorflow.keras.models import load_model
from keras.losses import binary_crossentropy
from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()
import scipy.stats as stats

os.environ['TF_CUDNN_DETERMINISTIC'] = 'true'
##os.environ['TF_DETERMINISTIC_OPS'] = 'true'
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.experimental.enable_op_determinism()  ### added
tf.compat.v1.enable_eager_execution() 

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0,1,2,3'


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Loading data (not important, lots of artifact code from previous experiments)

In [ ]:
##############################################
### loading data code 
##############################################


#Computer
#directory = "C:\\Users\\pookiee\\Desktop\\CNN CRT\\"
directory = '/root/DL_CRT_project/CNN_data/'

df_og = pd.read_csv(directory + 'CNN_CRT_data.csv')
df_NoImp = pd.read_csv(directory + 'CNN_CRT_data.csv')

reset_seeds(123)
def clean_data(dataframe, iaea):
    # filter by  death 
    dataframe = dataframe[np.array(dataframe['Death'] == 0)]
    
    # subset iaea
    
    ids = dataframe['ID']
    if iaea:
        length_checker = np.vectorize(len)
        index = length_checker(ids) >= 4
        dataframe = dataframe[index]

    
    
    response = dataframe['response']
    # Remove for high NA
    # Echo_vars, AF, CKD, Ischemia, Metroprolo
    categorical = ['ACEI_or_ARB','CABG','CAD','Concordance','DM','Gender','HTN','Hospitalization','LBBB', 'MI','NYHA','PCI','Race', 'ID', 'Smoking']

    continuous = [ 'ECG_pre_QRSd', 'SPECT_pre_LVEF', 'SPECT_pre_ESV', 'SPECT_pre_EDV','SPECT_pre_WTper',
              'SPECT_pre_PSD', 'SPECT_pre_PBW', 'Age','SPECT_pre_50scar', 
              'SPECT_pre_gMyoMass', 'SPECT_pre_WTsum', 'SPECT_pre_StrokeVolume', 'SPECT_pre_PhaseKurt',
              'SPECT_pre_Diastolic_PhasePeak', 'SPECT_pre_Diastolic_PhaseSkew','SPECT_pre_Diastolic_PBW' , 'SPECT_pre_Diastolic_PSD',
              'SPECT_pre_EDSI', 'SPECT_pre_EDE', 'SPECT_pre_ESSI', 'SPECT_pre_ESE', 'SPECT_pre_Diastolic_PhaseKurt', 'SPECT_pre_PhasePeak', 'SPECT_pre_SRscore']
    
    # Create continuous resonse 
    dataframe = dataframe.dropna(subset = 'SPECT_post_LVEF')
    response_vector = dataframe['SPECT_post_LVEF'] - dataframe['SPECT_pre_LVEF']
    
    # Remove post values 
    dataframe = pd.concat([dataframe[continuous],dataframe[categorical]], axis = 1)
    
    # Add response 
    dataframe.insert(0, 'resp', response_vector)
    dataframe.insert(0, 'response', response)

    # Drop NA
    dataframe = dataframe.dropna(how = 'any')
    #dataframe = dataframe.astype(int)
    response = dataframe['response']
    #id label for images 
    ids = dataframe['ID']
 
    del dataframe['response']
    del dataframe['ID']
       
    # dummify nyha and race
    nyha_race_dummy = pd.get_dummies(dataframe[['NYHA','Race']].astype(str))
    
    #Combine Dataframe with all features
    
    dataframe = pd.concat([nyha_race_dummy, dataframe], axis = 1)
    super_response = np.where(dataframe['resp'] >= 15, 1, 0)
    return dataframe, response, ids, super_response

df, response, ids, super_response = clean_data(df_og, False)
#color_data  = pd.get_dummies(response, prefix="response") # color discrete 
color_data  = df['resp']

                          
#del df['resp'] # the continuous normal response 

def norm(image_array):
    return image_array / 255

input_shape = 64
input_shape = (input_shape, input_shape)

def load_image(identification):
    perf_image_array = []
    wallthk_image_array = []
    systolic_image_array = []
    
    for idd in identification:
        patient_id = idd

        perf_path = directory + patient_id + '_perfusion.png'
        systolic_path = directory + patient_id + '_systolicPhase.png'
        wallthk_path = directory + patient_id + '_wallthk.png'

        perf_image = tf.keras.preprocessing.image.load_img(perf_path, color_mode = 'grayscale', target_size = input_shape)
        systolic_image = tf.keras.preprocessing.image.load_img(systolic_path, color_mode = 'grayscale', target_size = input_shape)
        wallthk_image = tf.keras.preprocessing.image.load_img(wallthk_path, color_mode = 'grayscale', target_size = input_shape)

        perf_image = tf.keras.preprocessing.image.img_to_array(perf_image, data_format = "channels_last")
        systolic_image = tf.keras.preprocessing.image.img_to_array(systolic_image, data_format = "channels_last")
        wallthk_image = tf.keras.preprocessing.image.img_to_array(wallthk_image, data_format = "channels_last")

        perf_image = norm(perf_image)
        systolic_image = norm(systolic_image)
        wallthk_image = norm(wallthk_image)
        
        perf_image_array.append(perf_image.reshape((1,64,64)))
        systolic_image_array.append(systolic_image.reshape((1,64,64)))
        wallthk_image_array.append(wallthk_image.reshape((1,64,64)))

    perf_conc = np.concatenate(perf_image_array, axis = 0)
    syst_conc = np.concatenate(systolic_image_array, axis = 0)
    wall_conc = np.concatenate(wallthk_image_array, axis = 0)

    conc_img = np.concatenate([perf_image_array, systolic_image_array, wallthk_image_array], axis = 1)
    print('max',np.max(conc_img))
    print('min',np.min(conc_img))
    conc_redo = np.concatenate([np.expand_dims(perf_conc, axis = 0),np.expand_dims(syst_conc, axis = 0), np.expand_dims(wall_conc, axis = 0)], axis = 0)

    return perf_image_array, systolic_image_array, wallthk_image_array, np.transpose(conc_img, [0,2,3,1]), np.transpose(conc_redo, [1,2,3,0])
perf, syst, wall, conc, conc2 = load_image(ids)

from gtda.images import Binarizer, RadialFiltration, ImageToPointCloud, DensityFiltration
from gtda.homology import VietorisRipsPersistence

tpe = ['Gray', 'TGray', 'bin', 'den', 'rips', 'rad']
def filtration(imgs, filterr, bin_threshold):
    
    if filterr == 'Gray':
        output = imgs
        
    elif filterr == 'TGray':
        new_img = []
        for i in range(len(imgs)):
            new_img.append(np.max(imgs[i]) - imgs[i])
        output = new_img 
        
    elif filterr == 'bin':
        binarizer = Binarizer(threshold=bin_threshold)
        output = binarizer.fit_transform(imgs)
        
    elif filterr == 'den':
        binarizer = Binarizer(threshold=bin_threshold)
        bin_img = binarizer.fit_transform(imgs)
        
        den_filter = DensityFiltration()
        output = den_filter.fit_transform(bin_img)
        
    elif filterr == 'rad':
        binarizer = Binarizer(threshold=bin_threshold)
        bin_img = binarizer.fit_transform(imgs)
        
        rad_filter = RadialFiltration(center = np.array([1,31,31]))
        output = rad_filter.fit_transform(bin_img)
    
    
    return output

def view_filt_img(imgs, index):
    imgs = np.array(imgs)
    print('Filtered Image')
    plt.imshow(np.squeeze(imgs[index,:,:,:]))
    plt.colorbar()
    plt.show()
    
    print('persistance Diagram')
    plt.imshow(np.squeeze(imgs[index+1,:,:,:]))
    plt.colorbar()
    plt.show()
          
# filt_img = filtration(perf, 'TGray', 0.7)
# view_filt_img(filt_img,0)

color_map = [
    [0.000000, 0.000000, 0.000000],
    [0.000000, 0.007843, 0.007843],
    [0.000000, 0.015686, 0.015686],
    [0.000000, 0.023529, 0.023529],
    [0.000000, 0.031373, 0.031373],
    [0.000000, 0.039216, 0.039216],
    [0.000000, 0.047059, 0.047059],
    [0.000000, 0.054902, 0.054902],
    [0.000000, 0.062745, 0.062745],
    [0.000000, 0.070588, 0.070588],
    [0.000000, 0.078431, 0.078431],
    [0.000000, 0.086275, 0.086275],
    [0.000000, 0.094118, 0.094118],
    [0.000000, 0.101961, 0.101961],
    [0.000000, 0.109804, 0.109804],
    [0.000000, 0.117647, 0.117647],
    [0.000000, 0.129412, 0.125490],
    [0.000000, 0.137255, 0.133333],
    [0.000000, 0.145098, 0.141176],
    [0.000000, 0.152941, 0.149020],
    [0.000000, 0.160784, 0.156863],
    [0.000000, 0.168627, 0.164706],
    [0.000000, 0.176471, 0.172549],
    [0.000000, 0.184314, 0.180392],
    [0.000000, 0.192157, 0.188235],
    [0.000000, 0.200000, 0.196078],
    [0.000000, 0.207843, 0.203922],
    [0.000000, 0.215686, 0.211765],
    [0.000000, 0.223529, 0.219608],
    [0.000000, 0.231373, 0.227451],
    [0.000000, 0.239216, 0.235294],
    [0.000000, 0.247059, 0.243137],
    [0.000000, 0.254902, 0.250980],
    [0.000000, 0.262745, 0.258824],
    [0.000000, 0.270588, 0.266667],
    [0.000000, 0.278431, 0.274510],
    [0.000000, 0.286275, 0.282353],
    [0.000000, 0.294118, 0.290196],
    [0.000000, 0.301961, 0.298039],
    [0.000000, 0.309804, 0.305882],
    [0.000000, 0.317647, 0.313725],
    [0.000000, 0.325490, 0.321569],
    [0.000000, 0.333333, 0.329412],
    [0.000000, 0.341176, 0.337255],
    [0.000000, 0.349020, 0.345098],
    [0.000000, 0.356863, 0.352941],
    [0.000000, 0.364706, 0.360784],
    [0.000000, 0.372549, 0.368627],
    [0.000000, 0.384314, 0.376471],
    [0.000000, 0.392157, 0.384314],
    [0.000000, 0.400000, 0.392157],
    [0.000000, 0.407843, 0.400000],
    [0.000000, 0.415686, 0.407843],
    [0.000000, 0.423529, 0.415686],
    [0.000000, 0.431373, 0.423529],
    [0.000000, 0.439216, 0.431373],
    [0.000000, 0.447059, 0.439216],
    [0.000000, 0.454902, 0.447059],
    [0.000000, 0.462745, 0.454902],
    [0.000000, 0.470588, 0.462745],
    [0.000000, 0.478431, 0.470588],
    [0.000000, 0.486275, 0.478431],
    [0.000000, 0.494118, 0.486275],
    [0.000000, 0.501961, 0.494118],
    [0.007843, 0.494118, 0.501961],
    [0.015686, 0.486275, 0.505882],
    [0.023529, 0.478431, 0.513725],
    [0.031373, 0.470588, 0.521569],
    [0.039216, 0.462745, 0.529412],
    [0.047059, 0.454902, 0.537255],
    [0.054902, 0.447059, 0.545098],
    [0.062745, 0.439216, 0.552941],
    [0.070588, 0.431373, 0.560784],
    [0.078431, 0.423529, 0.568627],
    [0.086275, 0.415686, 0.576471],
    [0.094118, 0.407843, 0.584314],
    [0.101961, 0.400000, 0.592157],
    [0.109804, 0.392157, 0.600000],
    [0.117647, 0.384314, 0.607843],
    [0.125490, 0.376471, 0.615686],
    [0.133333, 0.372549, 0.623529],
    [0.141176, 0.364706, 0.631373],
    [0.149020, 0.356863, 0.639216],
    [0.156863, 0.349020, 0.647059],
    [0.164706, 0.341176, 0.654902],
    [0.168627, 0.333333, 0.662745],
    [0.176471, 0.325490, 0.670588],
    [0.184314, 0.317647, 0.678431],
    [0.192157, 0.309804, 0.686275],
    [0.200000, 0.301961, 0.694118],
    [0.207843, 0.294118, 0.701961],
    [0.215686, 0.286275, 0.709804],
    [0.223529, 0.278431, 0.717647],
    [0.231373, 0.270588, 0.725490],
    [0.239216, 0.262745, 0.733333],
    [0.247059, 0.254902, 0.741176],
    [0.254902, 0.247059, 0.749020],
    [0.262745, 0.239216, 0.756863],
    [0.270588, 0.231373, 0.764706],
    [0.278431, 0.223529, 0.772549],
    [0.286275, 0.215686, 0.780392],
    [0.294118, 0.207843, 0.788235],
    [0.301961, 0.200000, 0.796078],
    [0.309804, 0.192157, 0.803922],
    [0.317647, 0.184314, 0.811765],
    [0.325490, 0.176471, 0.819608],
    [0.333333, 0.168627, 0.827451],
    [0.341176, 0.160784, 0.835294],
    [0.349020, 0.152941, 0.843137],
    [0.356863, 0.145098, 0.850980],
    [0.364706, 0.137255, 0.858824],
    [0.372549, 0.129412, 0.866667],
    [0.380392, 0.125490, 0.874510],
    [0.388235, 0.117647, 0.882353],
    [0.396078, 0.109804, 0.890196],
    [0.403922, 0.101961, 0.898039],
    [0.411765, 0.094118, 0.905882],
    [0.419608, 0.086275, 0.913725],
    [0.427451, 0.078431, 0.921569],
    [0.435294, 0.070588, 0.929412],
    [0.443137, 0.062745, 0.937255],
    [0.450980, 0.054902, 0.945098],
    [0.458824, 0.047059, 0.952941],
    [0.466667, 0.039216, 0.960784],
    [0.474510, 0.031373, 0.968627],
    [0.482353, 0.023529, 0.976471],
    [0.490196, 0.015686, 0.984314],
    [0.498039, 0.007843, 0.992157],
    [0.501961, 0.000000, 1.000000],
    [0.509804, 0.007843, 0.984314],
    [0.517647, 0.015686, 0.968627],
    [0.525490, 0.023529, 0.952941],
    [0.533333, 0.031373, 0.937255],
    [0.541176, 0.039216, 0.921569],
    [0.549020, 0.047059, 0.905882],
    [0.556863, 0.054902, 0.890196],
    [0.564706, 0.062745, 0.874510],
    [0.572549, 0.070588, 0.858824],
    [0.580392, 0.078431, 0.843137],
    [0.588235, 0.086275, 0.827451],
    [0.596078, 0.094118, 0.811765],
    [0.603922, 0.101961, 0.796078],
    [0.611765, 0.109804, 0.780392],
    [0.619608, 0.117647, 0.764706],
    [0.627451, 0.125490, 0.749020],
    [0.635294, 0.133333, 0.733333],
    [0.643137, 0.141176, 0.717647],
    [0.650980, 0.149020, 0.701961],
    [0.658824, 0.156863, 0.686275],
    [0.666667, 0.164706, 0.670588],
    [0.674510, 0.172549, 0.654902],
    [0.682353, 0.180392, 0.639216],
    [0.690196, 0.188235, 0.623529],
    [0.698039, 0.196078, 0.607843],
    [0.705882, 0.203922, 0.592157],
    [0.713725, 0.211765, 0.576471],
    [0.721569, 0.219608, 0.560784],
    [0.729412, 0.227451, 0.545098],
    [0.737255, 0.235294, 0.529412],
    [0.745098, 0.243137, 0.513725],
    [0.752941, 0.250980, 0.501961],
    [0.760784, 0.258824, 0.486275],
    [0.768627, 0.266667, 0.470588],
    [0.776471, 0.274510, 0.454902],
    [0.784314, 0.282353, 0.439216],
    [0.792157, 0.290196, 0.423529],
    [0.800000, 0.298039, 0.407843],
    [0.807843, 0.305882, 0.392157],
    [0.815686, 0.313725, 0.376471],
    [0.823529, 0.321569, 0.360784],
    [0.831373, 0.329412, 0.345098],
    [0.835294, 0.337255, 0.329412],
    [0.843137, 0.345098, 0.313725],
    [0.850980, 0.352941, 0.298039],
    [0.858824, 0.360784, 0.282353],
    [0.866667, 0.368627, 0.266667],
    [0.874510, 0.376471, 0.250980],
    [0.882353, 0.384314, 0.235294],
    [0.890196, 0.392157, 0.219608],
    [0.898039, 0.400000, 0.203922],
    [0.905882, 0.407843, 0.188235],
    [0.913725, 0.415686, 0.172549],
    [0.921569, 0.423529, 0.156863],
    [0.929412, 0.431373, 0.141176],
    [0.937255, 0.439216, 0.125490],
    [0.945098, 0.447059, 0.109804],
    [0.952941, 0.454902, 0.094118],
    [0.960784, 0.462745, 0.078431],
    [0.968627, 0.470588, 0.062745],
    [0.976471, 0.478431, 0.047059],
    [0.984314, 0.486275, 0.031373],
    [0.992157, 0.494118, 0.015686],
    [1.000000, 0.505882, 0.000000],
    [1.000000, 0.513725, 0.015686],
    [1.000000, 0.521569, 0.031373],
    [1.000000, 0.529412, 0.047059],
    [1.000000, 0.537255, 0.062745],
    [1.000000, 0.545098, 0.078431],
    [1.000000, 0.552941, 0.094118],
    [1.000000, 0.560784, 0.109804],
    [1.000000, 0.568627, 0.125490],
    [1.000000, 0.576471, 0.141176],
    [1.000000, 0.584314, 0.156863],
    [1.000000, 0.592157, 0.176471],
    [1.000000, 0.600000, 0.192157],
    [1.000000, 0.607843, 0.207843],
    [1.000000, 0.615686, 0.223529],
    [1.000000, 0.623529, 0.239216],
    [1.000000, 0.631373, 0.254902],
    [1.000000, 0.639216, 0.270588],
    [1.000000, 0.647059, 0.286275],
    [1.000000, 0.654902, 0.301961],
    [1.000000, 0.662745, 0.317647],
    [1.000000, 0.670588, 0.333333],
    [1.000000, 0.678431, 0.349020],
    [1.000000, 0.686275, 0.364706],
    [1.000000, 0.694118, 0.380392],
    [1.000000, 0.701961, 0.396078],
    [1.000000, 0.709804, 0.411765],
    [1.000000, 0.717647, 0.427451],
    [1.000000, 0.725490, 0.443137],
    [1.000000, 0.733333, 0.458824],
    [1.000000, 0.741176, 0.474510],
    [1.000000, 0.749020, 0.490196],
    [1.000000, 0.756863, 0.509804],
    [1.000000, 0.764706, 0.525490],
    [1.000000, 0.772549, 0.541176],
    [1.000000, 0.780392, 0.556863],
    [1.000000, 0.788235, 0.572549],
    [1.000000, 0.796078, 0.588235],
    [1.000000, 0.803922, 0.603922],
    [1.000000, 0.811765, 0.619608],
    [1.000000, 0.819608, 0.635294],
    [1.000000, 0.827451, 0.650980],
    [1.000000, 0.835294, 0.666667],
    [1.000000, 0.843137, 0.682353],
    [1.000000, 0.850980, 0.698039],
    [1.000000, 0.858824, 0.713725],
    [1.000000, 0.866667, 0.729412],
    [1.000000, 0.874510, 0.745098],
    [1.000000, 0.882353, 0.760784],
    [1.000000, 0.890196, 0.776471],
    [1.000000, 0.898039, 0.792157],
    [1.000000, 0.905882, 0.807843],
    [1.000000, 0.913725, 0.823529],
    [1.000000, 0.921569, 0.843137],
    [1.000000, 0.929412, 0.858824],
    [1.000000, 0.937255, 0.874510],
    [1.000000, 0.945098, 0.890196],
    [1.000000, 0.952941, 0.905882],
    [1.000000, 0.960784, 0.921569],
    [1.000000, 0.968627, 0.937255],
    [1.000000, 0.976471, 0.952941],
    [1.000000, 0.984314, 0.968627],
    [1.000000, 0.992157, 0.984314],
    [1.000000, 1.000000, 1.000000]]

def grayscale_to_rgb(grayscale_image, colormap):
    grayscale_image = np.array(grayscale_image)
    grayscale_image = grayscale_image * 255

    # Check that the grayscale image has values that range from 0 to 255
    # and that the colormap has a length of 256
    # if grayscale_image.min() < 0 or grayscale_image.max() > 255:
    #     raise ValueError('Grayscale values must be in the range 0-255')
    if len(colormap) != 256:
        raise ValueError('Colormap must have a length of 256')

    # Create an empty RGB image
    rgb_image = np.zeros((grayscale_image.shape[0], grayscale_image.shape[1], grayscale_image.shape[2], 3), dtype=np.uint8)

    # Iterate over the grayscale image and set the pixel values in the RGB image
    for k in range(grayscale_image.shape[0]):
        for i in range(grayscale_image.shape[1]):
            for j in range(grayscale_image.shape[2]):
                # Lookup the corresponding RGB value in the colormap
                r, g, b = colormap[int(grayscale_image[k,i, j, 0] )] 
                # Set the pixel value in the RGB image
                rgb_image[k,i, j, 0] = r * (int(grayscale_image[k,i, j, 0]))
                rgb_image[k,i, j, 1] = g * (int(grayscale_image[k,i, j, 0]))
                rgb_image[k,i, j, 2] = b * (int(grayscale_image[k, i, j, 0]))

  # Return the RGB image
    return rgb_image

# Defining functions for running main loop (training and evaluating models)

###  special rule (avg_prob - .5) <= std threshold, then automaticallys passes onto next stage + std of sample > std threshold, then passes onto next stage

In [ ]:
reset_seeds(123)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import RocCurveDisplay
from sklearn.linear_model import Perceptron


########## define models for hyperparameter search 
def tune_ml_svm(hp):
    reset_seeds(123)
    
    model = SVC(kernel = hp.Choice('kernel',['linear', 'rbf', 'poly', 'sigmoid'])
                , class_weight = "balanced", 
                random_state = 123, 
                probability = True,
               gamma = hp.Choice('gamma', ['scale', 'auto']),
               C = hp.Float('Cost', min_value = 0.01, max_value = 100, sampling = 'LOG', default = 1),
               degree = hp.Int('degree', 1, 3)
               ) 
  
    return model


def tune_ml_enet(hp):
    reset_seeds(123)

    model = LogisticRegression( penalty = 'elasticnet',
        C = hp.Float('Cost', min_value = 0.0001, max_value = 100, sampling = 'LOG', default = 1),
        l1_ratio = hp.Float('l1', min_value = 0, max_value = 1, sampling = 'linear', default = 1),
        solver = 'saga',
        random_state = 123,
        class_weight = "balanced",
        max_iter = 800
        )
  
    return model

def tune_ml_log(hp):
    reset_seeds(123)

    model = LogisticRegression( penalty = 'none',
        C = hp.Float('Cost', min_value = 0.0001, max_value = 100, sampling = 'LOG', default = 1),
        solver = 'saga',
        random_state = 123,
        class_weight = "balanced",
        max_iter = 800
        )
  
    return model

def tune_ml_rf(hp):
    reset_seeds(123)

    model = RandomForestClassifier(
        n_estimators=hp.Int('n_estimators', 10, 500, step=10),
        #max_depth= hp.Choice('max_depth', [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]),
        bootstrap = hp.Choice('bootstrap', [True, False]),
        min_samples_leaf = hp.Int('min_samples_leaf', 1, 5),
        min_samples_split = hp.Int('min_samples_split', 2, 5),
        max_features = hp.Choice('max_features', ['auto', 'sqrt', 'log2']),
        class_weight = "balanced",
        random_state = 123
        )
    
    return model


def tune_ml_ada(hp):
    reset_seeds(123)

    model = AdaBoostClassifier(DecisionTreeClassifier(criterion = hp.Choice('criterion', ['gini' , 'entropy' ]), 
                    class_weight = 'balanced', 
                    random_state = 123, 
                    splitter = hp.Choice('splitter', ['best', 'random']),
                    max_depth =  hp.Int('max_depth', 2, 20, step = 2 ),
                    max_features = hp.Choice('fts', ['auto', 'sqrt', 'log2']),
                    min_samples_leaf =  hp.Choice('min_leaf', [5, 10, 20, 50, 100])
                   ) ,n_estimators=hp.Int('n_estimators', 10, 200, step=10),
                              learning_rate= hp.Float('lr', min_value = 0.01, max_value = 100, sampling = 'LOG', default = 1),
                              random_state=123)
    return model



reset_seeds(123)
a = 2 ** np.arange(10)
a = a[1:9].tolist()

def find_correlation(df, thresh=0.9):
    """
    Given a numeric pd.DataFrame, this will find highly correlated features,
    and return a list of features to remove
    params:
    - df : pd.DataFrame
    - thresh : correlation threshold, will remove one of pairs of features with
               a correlation greater than this value
    """
    df = pd.DataFrame(df)
    corrMatrix = df.corr()
    corrMatrix.loc[:,:] =  np.tril(corrMatrix, k=-1)

    already_in = set()
    result = []

    for col in corrMatrix:
        perfect_corr = corrMatrix[col][corrMatrix[col] > thresh].index.tolist()
        if perfect_corr and col not in already_in:
            already_in.update(set(perfect_corr))
            perfect_corr.append(col)
            result.append(perfect_corr)


    select_nested = [f[1:] for f in result]
    select_flat = [i for j in select_nested for i in j]
    return select_flat

def weights(label):
    neg, pos = np.bincount(np.squeeze(label))
    total = neg + pos 
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weight = {0 : weight_for_0, 1: weight_for_1}
    return class_weight

def sensitivity(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def rfe_fs(clinical, label, automatic):
    # if automatic == 'percep':
    #     model = Perceptron(random_state=0)
    #     rfecv = RFECV(
    #         estimator=model,
    #         min_features_to_select=10,
    #         step=1,
    #         cv=7)
    #     fit = rfecv.fit(clinical, np.squeeze(label))

    # if automatic == 'rf':
    #     model = RandomForestClassifier(random_state=123)
    #     rfecv = RFECV(
    #         estimator=model,
    #         min_features_to_select=10,
    #         step=3,
    #         cv=4)
    #     fit = rfecv.fit(clinical, np.squeeze(label))

    if automatic == 'lr':
        model = LogisticRegression(solver='lbfgs',
        max_iter = 1000, random_state= 123)
        rfecv = RFECV(
            estimator=model,
            min_features_to_select=10,
            step=1,
            cv=7)
        fit = rfecv.fit(clinical, np.squeeze(label))

    return np.array(clinical[:,fit.support_.tolist()]), fit.support_.tolist()

class ml_CVTuner(keras_tuner.engine.tuner.Tuner):
    """
    Hyperparameter tuner for bootstrap ensemble models 
    """
    def run_trial(self, trial, x, y, **kwargs):
        reset_seeds(123)

        val_objective = []
        kwargs['sampling_fraction'] = trial.hyperparameters.Float('sampling_fraction', min_value = 0.7, max_value = .95, sampling = 'linear', default = 0.8)
        kwargs['n_models'] = trial.hyperparameters.Int('n_models', 25, 50, step = 3, default = 25)

        for train_indices, test_indices in StratifiedShuffleSplit(n_splits=5, test_size = 1/9,  random_state = 123).split(x, y):
            
            x_train = x[train_indices]
            x_test = x[test_indices]
                
            model = self.hypermodel.build(trial.hyperparameters)
            boot_model = Bootstraps(data_x = x_train, data_y = y[train_indices], n_models = kwargs['n_models'], model_itself = model , sampling_fraction = kwargs['sampling_fraction'] )
            boot_model.fit()
            #model.fit(x_train, y[train_indices])
            avg_probabilities, _, _, _ = boot_model.evaluate(test_data_x = x_test)

            #val_objective.append(metrics.accuracy_score(y[test_indices], model.predict(x_test)))
            val_objective.append(metrics.roc_auc_score(y[test_indices], avg_probabilities))
            del avg_probabilities, model, boot_model
            
        self.oracle.update_trial(trial.trial_id, {'val_acc': np.mean(val_objective)})

        

        
def train_evaluate(x_train2,x_train3 , y_train,x_test2,x_test3, y_test, train, test, fs_type, smote_bol, spatial, models, fold):
    reset_seeds(123)
    x_train_list = [ x_train2, x_train3]
    x_test_list = [x_test2,x_test3]
    
    val_data_x = []
    val_data_y = []
    
    val_data_x2 = []
    val_data_y2 = []

    test_data_x = [] 
    test_data_y = [] 

    feature_importances = [] 
    features_used = [] 
    
    auc_curves = []
    
    
    global staged_models
    staged_models = []
    ind_model_test = []
    boot_model_hyperParameters = []
    for ij, (x_train, x_test) in enumerate(zip(x_train_list,x_test_list)):
        print('On Stage: ' + str(ij) )
        
        #Standardize 
        scaler = preprocessing.StandardScaler().fit(x_train)
        xt = scaler.transform(x_train)
 
        print('y_train.shape', y_train.shape)
        print('xt.shape', xt.shape)

        # fs
        xt, fs_column = rfe_fs(xt, y_train, fs_type)
        x_test = x_test[:,fs_column]
        x_train = x_train[:,fs_column]


    ################################################################################################### Preprocess

        if smote_bol:
            oversample = SMOTE(random_state = 123)
            xx_train, yy_train = oversample.fit_resample(x_train, y_train)
        else:
            xx_train = x_train
            yy_train = y_train
            
        global class_weight
        class_weight = weights(yy_train)

        # Cor 
        corr_col = find_correlation(xx_train, .8)
        xx_train = np.delete(xx_train, corr_col, axis = 1)
        nzv = VarianceThreshold(0.01).fit(xx_train)
        xx_train = nzv.transform(xx_train)
        
        x_test = np.delete(x_test, corr_col, axis = 1)
        x_test = nzv.transform(x_test)
        
        #Standardize 
        scaler = preprocessing.StandardScaler().fit(xx_train)
        xx_train = scaler.transform(xx_train)
        x_test = scaler.transform(x_test)


        if spatial == True:
            #SPATIAL SIGN
            xx_train = preprocessing.normalize(xx_train, norm = 'l2')
            x_test = preprocessing.normalize(x_test, norm = 'l2')
            
        test_data_x.append(x_test)
        test_data_y.append(y_test)
        
        if ij == 0:
            print('stage 1 fs column', fs_column)
            buh = stage1_features[fs_column]
            print('stage 1 buh fs', buh)
            print('stage 1  cor', corr_col)
            buh = np.delete(np.array(buh), corr_col)
            print('stage 1 buh cor', buh)
            print('stage 1 nzv', np.logical_not(nzv.get_support()))
            print('stage 1 deleted',np.delete(buh, np.logical_not(nzv.get_support())))

            features_used.append(np.delete(buh, np.logical_not(nzv.get_support())).tolist())   
        elif ij == 1:
            print('stage 1 fs column', fs_column)

            yuh = stage2_features[fs_column]
            print('stage 1 buh fs', yuh)
            print('stage 1  cor', corr_col)
            yuh = np.delete(np.array(yuh), corr_col)
            print('stage 1 buh cor', yuh)
            print('stage 1 nzv', np.logical_not(nzv.get_support()))
            print('stage 1 deleted',np.delete(yuh, np.logical_not(nzv.get_support())))
            features_used.append(np.delete(yuh, np.logical_not(nzv.get_support())).tolist())  

####### sample a portion of train data for validation ########################################################################### 

        # first validation set 
        val_index = np.random.choice(np.arange(len(yy_train)), size = 20, replace = False).tolist() 
        train_index = [i for i in np.arange(len(yy_train)).tolist() if i not in val_index]

        x_val1 = xx_train[val_index]
        y_val1 = yy_train[val_index]
        
        x_ttrain = xx_train[train_index]
        y_ttrain = yy_train[train_index]

        print('on run: ',ij, ' with x_val1 shape: ', x_val1.shape )
        print('on run: ',ij, ' with x_train shape: ', x_ttrain.shape )

        val_data_x.append(x_val1)
        val_data_y.append(y_val1)
        print('x_val1  shape', x_val1.shape)
        print('y_val  shape', y_val1.shape)
        
        # second validation set 
        
        val_index = np.random.choice(np.arange(len(y_ttrain)), size = 20, replace = False).tolist() 
        train_index = [i for i in np.arange(len(y_ttrain)).tolist() if i not in val_index]

        x_val2 = x_ttrain[val_index]
        y_val2 = y_ttrain[val_index]
        
        x_ttrain = x_ttrain[train_index]
        y_ttrain = y_ttrain[train_index]

        print('on run: ',ij, ' with x_val2 shape: ', x_val2.shape )
        print('on run: ',ij, ' with x_train shape: ', x_ttrain.shape )

        val_data_x2.append(x_val2)
        val_data_y2.append(y_val2)
        print('x_val  shape', x_val2.shape)
        print('y_val  shape', y_val2.shape)
        
        
        
################################################################################################### individual bootstrap model hyperparameter tuning

        if models == 'enet': 
            tuner = ml_CVTuner(
            hypermodel = tune_ml_enet,
            oracle=keras_tuner.oracles.BayesianOptimizationOracle(
            objective= keras_tuner.Objective('val_acc', direction = 'max'),
            max_trials = 30,
            tune_new_entries = True,
            allow_new_entries = True,
            seed = 123),
            overwrite=True,
            directory = "CNN CRT",
            project_name = "keras_tuner")
            tuner.search(x_ttrain, y_ttrain)

        elif models =='log':
            tuner = ml_CVTuner(
            hypermodel = tune_ml_log,
            oracle=keras_tuner.oracles.BayesianOptimization(
            objective= keras_tuner.Objective('val_acc', direction = 'max'),
            max_trials = 1,
            tune_new_entries = True,
            allow_new_entries = True,
            seed = 123),
            overwrite=True,
            directory = "CNN CRT",
            project_name = "keras_tuner")
            tuner.search(x_ttrain, y_ttrain)

        elif models =='rf':
            print('on  rf tuner')
            tuner = ml_CVTuner(
            hypermodel = tune_ml_rf,
            oracle=keras_tuner.oracles.BayesianOptimization(
            objective= keras_tuner.Objective('val_acc', direction = 'max'),
            max_trials = 50,
            tune_new_entries = True,
            allow_new_entries = True,
            seed = 123),
            overwrite=True,
            directory = "CNN CRT",
            project_name = "keras_tuner")
            tuner.search(x_ttrain, y_ttrain)

        elif models == 'svm':
            tuner = ml_CVTuner(
            hypermodel = tune_ml_svm,
            oracle=keras_tuner.oracles.BayesianOptimization(
            objective= keras_tuner.Objective('val_acc', direction = 'max'),
            max_trials =  50,
            tune_new_entries = True,
            allow_new_entries = True,
            seed = 123),
            overwrite=True,
            directory = "CNN CRT",
            project_name = "keras_tuner")
            tuner.search(x_ttrain, y_ttrain)


        elif models == 'nb':
            tuner = ml_CVTuner(
            hypermodel = tune_ml_nb,
            oracle=keras_tuner.oracles.BayesianOptimization(
            objective= keras_tuner.Objective('val_acc', direction = 'max'),
            max_trials = 70,
            tune_new_entries = True,
            allow_new_entries = True,
            seed = 123),
            overwrite=True,
            directory = "CNN CRT",
            project_name = "keras_tuner")  
            tuner.search(x_ttrain, y_ttrain)

        elif models == 'ada':
            tuner = ml_CVTuner(
            hypermodel = tune_ml_ada,
            oracle=keras_tuner.oracles.BayesianOptimization(
            objective= keras_tuner.Objective('val_acc', direction = 'max'),
            max_trials = 70,
            tune_new_entries = True,
            allow_new_entries = True,
            seed = 123),
            overwrite=True,
            directory = "CNN CRT",
            project_name = "keras_tuner")    
        
            tuner.search(x_ttrain, y_ttrain)

    
################################################################################################### Fit outer bootstrap  model
        if models == 'enet': 
            model = LogisticRegression( penalty = 'elasticnet',
                C = tuner.get_best_hyperparameters(num_trials=1)[0].get('Cost'),
                l1_ratio = tuner.get_best_hyperparameters(num_trials=1)[0].get('l1'),
                solver = 'saga',
                random_state = 123,
                class_weight = "balanced",
                max_iter = 800
                )
            #model.fit(x_train, y_train)
        elif models == 'log': 
            model = LogisticRegression( penalty = 'none',
                C = tuner.get_best_hyperparameters(num_trials=1)[0].get('Cost'),
                solver = 'saga',
                random_state = 123,
                class_weight = "balanced",
                max_iter = 800
                )
            #model.fit(x_train, y_train)

        elif models =='rf':
            print('on fitting rf with tuned parameters')
            model = RandomForestClassifier(
                n_estimators= tuner.get_best_hyperparameters(num_trials=1)[0].get('n_estimators'),
                #max_depth= hp.Choice('max_depth', [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]),
                bootstrap = tuner.get_best_hyperparameters(num_trials=1)[0].get('bootstrap'),
                min_samples_leaf = tuner.get_best_hyperparameters(num_trials=1)[0].get('min_samples_leaf'),
                min_samples_split = tuner.get_best_hyperparameters(num_trials=1)[0].get('min_samples_split'),
                max_features = tuner.get_best_hyperparameters(num_trials=1)[0].get('max_features'),
                class_weight = 'balanced',
                random_state = 123
                )
            #model.fit(x_train, y_train)
            
        elif models == 'svm':
                
            model = SVC(kernel = tuner.get_best_hyperparameters(num_trials=1)[0].get('kernel'), 
                    class_weight = 'balanced', 
                    random_state = 123, 
                    probability = True,
                gamma = tuner.get_best_hyperparameters(num_trials=1)[0].get('gamma'),
                C =  tuner.get_best_hyperparameters(num_trials=1)[0].get('Cost'),
                degree = tuner.get_best_hyperparameters(num_trials=1)[0].get('degree')
                ) 
            
            #model.fit(x_train, y_train)

        elif models == 'nb':
            model = GaussianNB(var_smoothing =tuner.get_best_hyperparameters(num_trials=1)[0].get('smooth'))

        elif models == 'ada':
            model = AdaBoostClassifier(DecisionTreeClassifier(criterion = tuner.get_best_hyperparameters(num_trials=1)[0].get('criterion'), 
                        class_weight = 'balanced', 
                        random_state = 123, 
                        splitter = tuner.get_best_hyperparameters(num_trials=1)[0].get('splitter'),
                        max_depth =  tuner.get_best_hyperparameters(num_trials=1)[0].get('max_depth'),
                        max_features = tuner.get_best_hyperparameters(num_trials=1)[0].get('fts'),
                        min_samples_leaf =  tuner.get_best_hyperparameters(num_trials=1)[0].get('min_leaf')
                    ) ,n_estimators=tuner.get_best_hyperparameters(num_trials=1)[0].get('n_estimators'),
                                learning_rate= tuner.get_best_hyperparameters(num_trials=1)[0].get('lr'),
                                random_state=123)
            #model.fit(x_train, y_train)





###################################################################################################
###### fitting and bootstrap models for different stages
###################################################################################################

        boot_model = Bootstraps(data_x = x_ttrain, data_y = y_ttrain, n_models = tuner.get_best_hyperparameters(num_trials=1)[0].get('n_models'), 
                                model_itself = model , sampling_fraction = tuner.get_best_hyperparameters(num_trials=1)[0].get('sampling_fraction'))
        boot_model.fit()
        avg_probabilities, avg_class, std, num_majority_class = boot_model.evaluate(test_data_x = x_test)
        eval = [metrics.roc_auc_score(y_test, avg_probabilities), metrics.accuracy_score(y_test, avg_class), metrics.cohen_kappa_score(y_test, avg_class), metrics.recall_score(y_test, avg_class), metrics.recall_score(np.logical_not(y_test), np.logical_not(avg_class))]
        ind_model_test.append(eval)

        staged_models.append(boot_model)


        boot_model_hyperParameters.append([tuner.get_best_hyperparameters(num_trials=1)[0].get('n_models'), tuner.get_best_hyperparameters(num_trials=1)[0].get('sampling_fraction')])
    
        ft_importance = boot_model.return_importance()
        feature_importances.append(ft_importance)
        
        
        #### stage 1 and stage 2 auc plots 
        
        fig, ax = plt.subplots(figsize=(10, 10))
        viz = RocCurveDisplay.from_predictions(
            y_true = y_test,
            y_pred = avg_probabilities,
            name=f"CV Fold {fold + 1}",
            alpha=0.3,
            lw=1,
            ax=ax)
        
        auc_curves.append(viz)
        
        
        
        
        
###################################################################################################
###### tuning and evaluating multi-stage sequential model
###################################################################################################

    val_aucs = []
    tuners = []
    weightss =  [0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 15, 20]
    for scaling_func_weight in weightss:
        
        global stg0_weight
        stg0_weight = scaling_func_weight
        tuner = sequential_Tuner(
                    hypermodel = create_sequential_model,
                    oracle=keras_tuner.oracles.BayesianOptimization(
                    objective= keras_tuner.Objective('val_acc', direction = 'max'),
                    max_trials = 80,
                    tune_new_entries = True,
                    allow_new_entries = True,
                    seed = 123),
                    overwrite=True,
                    directory = "CNN CRT buh",
                    project_name = "keras_tuner")    


        tuner.search(val_data_x, val_data_y)
        tuners.append(tuner)


        sequential_model = build_sequential_model(std1_thresh = tuner.get_best_hyperparameters(num_trials=1)[0].get('std1_thresh'),
                                                halfway_thresh = tuner.get_best_hyperparameters(num_trials=1)[0].get('halfway_thresh'),
                                                model_list = staged_models)
        
        probs, stds_val, stg_transitions_val = sequential_model.evaluate(val_data_x2)
        val_aucs.append(metrics.roc_auc_score(val_data_y2[0], probs))
        
        
    # create best sequential model     
    best_performing_scl_func_weight_index = val_aucs.index(max(val_aucs))
    tuner = tuners[best_performing_scl_func_weight_index]
    
    scaling_weight = weightss[best_performing_scl_func_weight_index]
    
    sequential_model = build_sequential_model(std1_thresh = tuner.get_best_hyperparameters(num_trials=1)[0].get('std1_thresh'),
                                            halfway_thresh = tuner.get_best_hyperparameters(num_trials=1)[0].get('halfway_thresh'),
                                            model_list = staged_models)
        
        
    _, stds_val, stg_transitions_val = sequential_model.evaluate(val_data_x2)

        
    # evaluate on test set 
    probs, stds, stg_transitions = sequential_model.evaluate(test_data_x)

    clas = probs.round()

    performance = []
    y_test = test_data_y[0]
    performance.append(metrics.roc_auc_score(y_test, probs))
    performance.append(metrics.accuracy_score(y_test, clas))
    performance.append(metrics.cohen_kappa_score(y_test, clas))
    performance.append(metrics.recall_score(y_test, clas))
    performance.append(metrics.recall_score(np.logical_not(y_test), np.logical_not(clas)))
    
    ############# AUC plots 
    
    fig, ax = plt.subplots(figsize=(10, 10))
    viz = RocCurveDisplay.from_predictions(
        y_true = y_test,
        y_pred = probs,
        name=f"CV Fold {fold + 1}",
        alpha=0.3,
        lw=1,
        ax=ax)
    
    
    auc_curves.append(viz)

    

    return performance, ind_model_test, sequential_model, stds, boot_model_hyperParameters, stg_transitions, stds_val, stg_transitions_val, scaling_weight, feature_importances, auc_curves, features_used




###################################################################################################
###### sequential mutli stage hyperparameter tuning functions
###################################################################################################
def create_sequential_model(hp):
    """
    used to initialize model with specific hyperparameter configuration for search space
    """
   
    std1_thresh = hp.Float('std1_thresh', min_value = 0.01, max_value = 0.2, sampling = 'log', default = 0.1) # WAS .01 AJD .2 
    halfway_thresh = hp.Float('halfway_thresh', min_value = 0.02, max_value = 0.1, sampling = 'linear', default = 0.05)
    model = build_sequential_model(std1_thresh,  halfway_thresh, staged_models )

    return model 

def weighted_func_new2(weight):
    """
    weight - the weight hyperparameter
    scaling function applied to fraction of samples retained for first model (penalizes all predictions occuring with all data modalities (ecg + spect))
    """
    return 1 - (1/(math.exp(stg0_weight * weight) + 1))


class sequential_Tuner(keras_tuner.engine.tuner.Tuner):
    """
    Defines hyperparameter tuning model for finding multi-stage hyperparameters (std threshold and halfway_thresh)
    """
    def run_trial(self, trial, x, y, **kwargs):
        reset_seeds(123)
            
        model = self.hypermodel.build(trial.hyperparameters)
        probs, _, stg_trans = model.evaluate(x)

        # compute penalization for std thresholds 
        
        stg1 = stg_trans[0].sum() # number kept during stage 1 
            
        AUC = metrics.roc_auc_score(y[0], probs)
        
        weight = weighted_func_new2(stg1 / len(probs))
        
        weighted_auc = AUC * weight
        
        
        self.oracle.update_trial(trial.trial_id, {'val_acc': weighted_auc})

# modified for feature names 

In [ ]:
reset_seeds(123)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import RocCurveDisplay
from sklearn.linear_model import Perceptron


########## define models for hyperparameter search 
def tune_ml_svm(hp):
    reset_seeds(123)
    
    model = SVC(kernel = hp.Choice('kernel',['linear', 'rbf', 'poly', 'sigmoid'])
                , class_weight = "balanced", 
                random_state = 123, 
                probability = True,
               gamma = hp.Choice('gamma', ['scale', 'auto']),
               C = hp.Float('Cost', min_value = 0.01, max_value = 100, sampling = 'LOG', default = 1),
               degree = hp.Int('degree', 1, 3)
               ) 
  
    return model


def tune_ml_enet(hp):
    reset_seeds(123)

    model = LogisticRegression( penalty = 'elasticnet',
        C = hp.Float('Cost', min_value = 0.0001, max_value = 100, sampling = 'LOG', default = 1),
        l1_ratio = hp.Float('l1', min_value = 0, max_value = 1, sampling = 'linear', default = 1),
        solver = 'saga',
        random_state = 123,
        class_weight = "balanced",
        max_iter = 800
        )
  
    return model

def tune_ml_log(hp):
    reset_seeds(123)

    model = LogisticRegression( penalty = 'none',
        C = hp.Float('Cost', min_value = 0.0001, max_value = 100, sampling = 'LOG', default = 1),
        solver = 'saga',
        random_state = 123,
        class_weight = "balanced",
        max_iter = 800
        )
  
    return model

def tune_ml_rf(hp):
    reset_seeds(123)

    model = RandomForestClassifier(
        n_estimators=hp.Int('n_estimators', 10, 500, step=10),
        #max_depth= hp.Choice('max_depth', [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200]),
        bootstrap = hp.Choice('bootstrap', [True, False]),
        min_samples_leaf = hp.Int('min_samples_leaf', 1, 5),
        min_samples_split = hp.Int('min_samples_split', 2, 5),
        max_features = hp.Choice('max_features', ['auto', 'sqrt', 'log2']),
        class_weight = "balanced",
        random_state = 123
        )
    
    return model


def tune_ml_ada(hp):
    reset_seeds(123)

    model = AdaBoostClassifier(DecisionTreeClassifier(criterion = hp.Choice('criterion', ['gini' , 'entropy' ]), 
                    class_weight = 'balanced', 
                    random_state = 123, 
                    splitter = hp.Choice('splitter', ['best', 'random']),
                    max_depth =  hp.Int('max_depth', 2, 20, step = 2 ),
                    max_features = hp.Choice('fts', ['auto', 'sqrt', 'log2']),
                    min_samples_leaf =  hp.Choice('min_leaf', [5, 10, 20, 50, 100])
                   ) ,n_estimators=hp.Int('n_estimators', 10, 200, step=10),
                              learning_rate= hp.Float('lr', min_value = 0.01, max_value = 100, sampling = 'LOG', default = 1),
                              random_state=123)
    return model



reset_seeds(123)
a = 2 ** np.arange(10)
a = a[1:9].tolist()

def find_correlation(df, thresh=0.9):
    """
    Given a numeric pd.DataFrame, this will find highly correlated features,
    and return a list of features to remove
    params:
    - df : pd.DataFrame
    - thresh : correlation threshold, will remove one of pairs of features with
               a correlation greater than this value
    """
    df = pd.DataFrame(df)
    corrMatrix = df.corr()
    corrMatrix.loc[:,:] =  np.tril(corrMatrix, k=-1)

    already_in = set()
    result = []

    for col in corrMatrix:
        perfect_corr = corrMatrix[col][corrMatrix[col] > thresh].index.tolist()
        if perfect_corr and col not in already_in:
            already_in.update(set(perfect_corr))
            perfect_corr.append(col)
            result.append(perfect_corr)


    select_nested = [f[1:] for f in result]
    select_flat = [i for j in select_nested for i in j]
    return select_flat

def weights(label):
    neg, pos = np.bincount(np.squeeze(label))
    total = neg + pos 
    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weight = {0 : weight_for_0, 1: weight_for_1}
    return class_weight

def sensitivity(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def rfe_fs(clinical, label, automatic):
    # if automatic == 'percep':
    #     model = Perceptron(random_state=0)
    #     rfecv = RFECV(
    #         estimator=model,
    #         min_features_to_select=10,
    #         step=1,
    #         cv=7)
    #     fit = rfecv.fit(clinical, np.squeeze(label))

    # if automatic == 'rf':
    #     model = RandomForestClassifier(random_state=123)
    #     rfecv = RFECV(
    #         estimator=model,
    #         min_features_to_select=10,
    #         step=3,
    #         cv=4)
    #     fit = rfecv.fit(clinical, np.squeeze(label))

    if automatic == 'lr':
        model = LogisticRegression(solver='lbfgs',
        max_iter = 1000, random_state= 123)
        rfecv = RFECV(
            estimator=model,
            min_features_to_select=10,
            step=1,
            cv=7)
        fit = rfecv.fit(clinical, np.squeeze(label))

    return np.array(clinical[:,fit.support_.tolist()]), fit.support_.tolist()

class ml_CVTuner(keras_tuner.engine.tuner.Tuner):
    """
    Hyperparameter tuner for bootstrap ensemble models 
    """
    def run_trial(self, trial, x, y, **kwargs):
        reset_seeds(123)

        val_objective = []
        kwargs['sampling_fraction'] = trial.hyperparameters.Float('sampling_fraction', min_value = 0.7, max_value = .95, sampling = 'linear', default = 0.8)
        kwargs['n_models'] = trial.hyperparameters.Int('n_models', 25, 50, step = 3, default = 25)

        for train_indices, test_indices in StratifiedShuffleSplit(n_splits=5, test_size = 1/9,  random_state = 123).split(x, y):
            
            x_train = x[train_indices]
            x_test = x[test_indices]
                
            model = self.hypermodel.build(trial.hyperparameters)
            boot_model = Bootstraps(data_x = x_train, data_y = y[train_indices], n_models = kwargs['n_models'], model_itself = model , sampling_fraction = kwargs['sampling_fraction'] )
            boot_model.fit()
            #model.fit(x_train, y[train_indices])
            avg_probabilities, _, _, _ = boot_model.evaluate(test_data_x = x_test)

            #val_objective.append(metrics.accuracy_score(y[test_indices], model.predict(x_test)))
            val_objective.append(metrics.roc_auc_score(y[test_indices], avg_probabilities))
            del avg_probabilities, model, boot_model
            
        self.oracle.update_trial(trial.trial_id, {'val_acc': np.mean(val_objective)})

import operator

        
def train_evaluate(x_train2,x_train3 , y_train,x_test2,x_test3, y_test, train, test, fs_type, smote_bol, spatial, models, fold):
    reset_seeds(123)
    x_train_list = [ x_train2, x_train3]
    x_test_list = [x_test2,x_test3]
    
    val_data_x = []
    val_data_y = []
    
    val_data_x2 = []
    val_data_y2 = []

    test_data_x = [] 
    test_data_y = [] 

    feature_importances = [] 
    features_used = [] 
    
    auc_curves = []
    
    
    global staged_models
    staged_models = []
    ind_model_test = []
    boot_model_hyperParameters = []
    for ij, (x_train, x_test) in enumerate(zip(x_train_list,x_test_list)):
        print('On Stage: ' + str(ij) )
        
        #Standardize 
        scaler = preprocessing.StandardScaler().fit(x_train)
        xt = scaler.transform(x_train)
 
        print('y_train.shape', y_train.shape)
        print('xt.shape', xt.shape)

        # fs
        xt, fs_column = rfe_fs(xt, y_train, fs_type)
        x_test = x_test[:,fs_column]
        x_train = x_train[:,fs_column]


    ################################################################################################### Preprocess

        if smote_bol:
            oversample = SMOTE(random_state = 123)
            xx_train, yy_train = oversample.fit_resample(x_train, y_train)
        else:
            xx_train = x_train
            yy_train = y_train
            
        global class_weight
        class_weight = weights(yy_train)

        # Cor 
        corr_col = find_correlation(xx_train, .8)
        xx_train = np.delete(xx_train, corr_col, axis = 1)
        nzv = VarianceThreshold(0.01).fit(xx_train)
        xx_train = nzv.transform(xx_train)
        
        x_test = np.delete(x_test, corr_col, axis = 1)
        x_test = nzv.transform(x_test)
        
        #Standardize 
        scaler = preprocessing.StandardScaler().fit(xx_train)
        xx_train = scaler.transform(xx_train)
        x_test = scaler.transform(x_test)


        if spatial == True:
            #SPATIAL SIGN
            xx_train = preprocessing.normalize(xx_train, norm = 'l2')
            x_test = preprocessing.normalize(x_test, norm = 'l2')
            
        test_data_x.append(x_test)
        test_data_y.append(y_test)

        if ij == 0:
            print('stage 1 fs column', fs_column)
            buh = stage1_features[fs_column]
            print('stage 1 buh fs', buh)
            print('stage 1  cor', corr_col)
            buh = np.delete(np.array(buh), corr_col)
            print('stage 1 buh cor', buh)
            print('stage 1 nzv', np.logical_not(nzv.get_support()))
            print('stage 1 deleted',np.delete(buh, np.logical_not(nzv.get_support())))

            features_used.append(np.delete(buh, np.logical_not(nzv.get_support())).tolist())   
        elif ij == 1:
            print('stage 1 fs column', fs_column)

            yuh = stage2_features[fs_column]
            print('stage 1 buh fs', yuh)
            print('stage 1  cor', corr_col)
            yuh = np.delete(np.array(yuh), corr_col)
            print('stage 1 buh cor', yuh)
            print('stage 1 nzv', np.logical_not(nzv.get_support()))
            print('stage 1 deleted',np.delete(yuh, np.logical_not(nzv.get_support())))
            features_used.append(np.delete(yuh, np.logical_not(nzv.get_support())).tolist()) 
    

    return features_used




###################################################################################################
###### sequential mutli stage hyperparameter tuning functions
###################################################################################################
def create_sequential_model(hp):
    """
    used to initialize model with specific hyperparameter configuration for search space
    """
   
    std1_thresh = hp.Float('std1_thresh', min_value = 0.01, max_value = 0.2, sampling = 'log', default = 0.1) # WAS .01 AJD .2 
    halfway_thresh = hp.Float('halfway_thresh', min_value = 0.02, max_value = 0.1, sampling = 'linear', default = 0.05)
    model = build_sequential_model(std1_thresh,  halfway_thresh, staged_models )

    return model 

def weighted_func_new2(weight):
    """
    weight - the weight hyperparameter
    scaling function applied to fraction of samples retained for first model (penalizes all predictions occuring with all data modalities (ecg + spect))
    """
    return 1 - (1/(math.exp(stg0_weight * weight) + 1))


class sequential_Tuner(keras_tuner.engine.tuner.Tuner):
    """
    Defines hyperparameter tuning model for finding multi-stage hyperparameters (std threshold and halfway_thresh)
    """
    def run_trial(self, trial, x, y, **kwargs):
        reset_seeds(123)
            
        model = self.hypermodel.build(trial.hyperparameters)
        probs, _, stg_trans = model.evaluate(x)

        # compute penalization for std thresholds 
        
        stg1 = stg_trans[0].sum() # number kept during stage 1 
            
        AUC = metrics.roc_auc_score(y[0], probs)
        
        weight = weighted_func_new2(stg1 / len(probs))
        
        weighted_auc = AUC * weight
        
        
        self.oracle.update_trial(trial.trial_id, {'val_acc': weighted_auc})

In [ ]:
reset_seeds(123)
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.ensemble import BaggingClassifier
import time 
from sklearn.base import clone
import pickle

num_class = 2

stage_1 = ['NYHA_2', 'NYHA_3', 'NYHA_4', 'Race_1', 'Race_2', 'Race_3', 'Race_4','Race_5', 'Age',  'ACEI_or_ARB', 'CABG', 'CAD', 'DM','Gender', 'HTN',  'MI', 'PCI', 'Smoking']
stage_2 = ['LBBB', 'ECG_pre_QRSd' ]
stage_3 = ['SPECT_pre_LVEF', 'SPECT_pre_ESV','SPECT_pre_EDV', 'SPECT_pre_WTper', 'SPECT_pre_PSD', 'SPECT_pre_PBW', 'SPECT_pre_50scar', 'SPECT_pre_gMyoMass', 'SPECT_pre_WTsum',
            'SPECT_pre_StrokeVolume', 'SPECT_pre_PhaseKurt','SPECT_pre_Diastolic_PhasePeak', 'SPECT_pre_Diastolic_PhaseSkew','SPECT_pre_Diastolic_PBW', 'SPECT_pre_Diastolic_PSD', 'SPECT_pre_EDSI',
            'SPECT_pre_EDE', 'SPECT_pre_ESSI', 'SPECT_pre_ESE','SPECT_pre_Diastolic_PhaseKurt', 'SPECT_pre_PhasePeak', 'PECT_pre_SRscore',  'Concordance']


class Bootstraps:
    """
    n_models - # of models in ensemble
    model_itself - base model to use (RF, svm, enet, etc...) 
    sampling_fraction - fraction of samples for psuedo-bootstrap
    
    """
    def __init__(self, data_x, data_y, n_models, model_itself, sampling_fraction):
        self.data_x = data_x
        self.data_y = data_y
        self.n_models = n_models
        self.model_itself = model_itself
        self.sampling_fraction = sampling_fraction
        self.list_of_models = []

    def fit(self): 
        fitted_models = []
        for num in range(self.n_models):
            reset_seeds(123 + num)  
            model = clone(self.model_itself)
            np.random.seed(123 + num) 

            resample_index = np.random.choice(np.arange(len(self.data_y)), size = round(self.sampling_fraction * len(self.data_y)), replace = True).tolist()

            model.fit(self.data_x[resample_index], self.data_y[resample_index]);
            fitted_models.append(model)
            
            del model, resample_index
        
        self.list_of_models = fitted_models
        #return fitted_models
    
    def evaluate(self, test_data_x):
        probabilities = []
        for trained_model in self.list_of_models:
            prob_predictions = trained_model.predict_proba(test_data_x)
            probabilities.append(prob_predictions[:,1])
        
        probabilities = np.array(probabilities)
        avg_probabilities = probabilities.mean(axis = 0)
        avg_class = np.where(avg_probabilities >= 0.50, 1, 0)
        std = probabilities.std(axis = 0)
        
        class_counts = probabilities.round()
        num_response = np.sum(class_counts, axis = 0 )
        num_majority_class = np.where(num_response <= self.n_models/2, self.n_models - num_response, num_response)
        
        return avg_probabilities, avg_class, std, num_majority_class
    
    def return_importance(self):
        importance = [] 
        for trained_model in self.list_of_models:
            result = permutation_importance(trained_model,  self.data_x, self.data_y, n_repeats=10,
                                 random_state=123)
            importance.append(result.importances_mean)
            
        importance = np.array(importance).mean(axis = 0)
        return importance
    

class build_sequential_model():
    """
    std1_thresh - for deciding rules
    halfway_thresh - for deciding rules
    model_list - list of length 2 for the 2 stages which contains the ensemble models
    """
    def __init__(self, std1_thresh,halfway_thresh, model_list):
        
        self.std1_thresh = std1_thresh
        self.model_list = model_list
        self.halfway_thresh = halfway_thresh
        
    def evaluate(self, test_data_list):
        """
        aggregated_probabilities = final probabilities which stem from either the first or second model based off the rules
        keep = whether or not the sample should be predicted using the first model ( 1 = predict using first model, i.e., the sample did not need spect data)
        """
        print('test_data_list[0].shape',test_data_list[0].shape)
        aggregated_probabilities = np.zeros(shape = test_data_list[0].shape[0])
        keep = []
        standard_deviations = []
        for stage, (modell, data) in enumerate(zip(self.model_list, test_data_list)):
            print('stage', stage, 'date shape', data.shape)
            if stage == 0:
                avg_probabilities, _, std, _ = modell.evaluate(data)
                keep.append(np.where(np.logical_and(std <= self.std1_thresh, abs(avg_probabilities - 0.5) > self.halfway_thresh), 1, 0))
                aggregated_probabilities += np.where(keep[0] == 1, avg_probabilities, 0)
                standard_deviations.append(std)
                
            elif stage == 1:
                avg_probabilities, _, std, _ = modell.evaluate(data)
                aggregated_probabilities += np.where(keep[0] == 1, 0, avg_probabilities)
                standard_deviations.append(std)

        return aggregated_probabilities, standard_deviations, keep
    
    
stage_1 = ['NYHA_2', 'NYHA_3', 'NYHA_4', 'Race_1', 'Race_2', 'Race_3', 'Race_4','Race_5', 'Age',  'ACEI_or_ARB', 'CABG', 'CAD', 'DM','Gender', 'HTN',  'MI', 'PCI', 'Smoking']
stage_2 = ['LBBB', 'ECG_pre_QRSd' ]
stage_3 = ['SPECT_pre_LVEF', 'SPECT_pre_ESV','SPECT_pre_EDV', 'SPECT_pre_WTper', 'SPECT_pre_PSD', 'SPECT_pre_PBW', 'SPECT_pre_50scar', 'SPECT_pre_gMyoMass', 'SPECT_pre_WTsum',
            'SPECT_pre_StrokeVolume', 'SPECT_pre_PhaseKurt','SPECT_pre_Diastolic_PhasePeak', 'SPECT_pre_Diastolic_PhaseSkew','SPECT_pre_Diastolic_PBW', 'SPECT_pre_Diastolic_PSD', 'SPECT_pre_EDSI',
            'SPECT_pre_EDE', 'SPECT_pre_ESSI', 'SPECT_pre_ESE','SPECT_pre_Diastolic_PhaseKurt', 'SPECT_pre_PhasePeak', 'PECT_pre_SRscore',  'Concordance']




############################## Call experiment via this script ##############################

def score(dataframe,  save_path,fs_method, smotee, spatial_sign, what_model  ):
    """
   Main function for running experiement 
    fs_method - feature selection method 
    smotee - if to use SMOTE
    spatial_sign - if to use spatial sign 
    what_model - what base model to use (RF, svm, etc...)
    """

    clinical_data = dataframe.drop(columns = ['resp', 'Race', 'NYHA', 'Hospitalization'])
    
    
    # making data for different stages 
    X2 = np.array(clinical_data.filter(stage_1 + stage_2))
    X3 = np.array(clinical_data.filter(stage_1 + stage_2 + stage_3))
 
    # define global features 
    global stage1_features
    stage1_features = clinical_data.filter(stage_1 + stage_2).columns
 
    global stage2_features 
    stage2_features = clinical_data.filter(stage_1 + stage_2 + stage_3).columns
    
    # foprmat data
    Y = np.where(np.array(response)  == 'yes', 1, 0)    
   
    # evaluate ML
    stage0_wt = [1] # weight parameter for scaling function for weighted auc 
    stage1_wt = [1]
    for weights in product(stage0_wt, stage1_wt):

        combo_output = []
        scores = np.zeros((10,5))
        idx = 0
        ind_scores = [] 
        sequential_mods = []
        stand_dev = []
        boot_model_hypers = []
        stage_transitions = []
        val_stand_dev = []
        val_stage_transitions = []
        scle_weights = []
        ft_import = []
        auc_curves = []
        ft_used = []
        crnt_fold = 0 
        for train, test in StratifiedKFold(n_splits=10, shuffle = True, random_state = 123).split(X2, Y):
            print(len(train))
            print('Fold', idx)
            features = train_evaluate( X2[train], X3[train], Y[train],  X2[test], X3[test], Y[test], train, test, fs_type = fs_method, smote_bol = smotee , spatial = spatial_sign, models = what_model, fold =  crnt_fold)
            ft_used.append(features)
            idx += 1
            crnt_fold += 1
        #scores.to_csv(save_path , mode='a', header=not os.path.exists(save_path ), index = False)
    
        data_run = [ ft_used]
   
        with open(what_model + '_' +  str(1) + '_' +  str(1) + 'features', "wb") as fp:   #Pickling
            pickle.dump(data_run, fp)
   
   

# Defining bootstrap ensemble and multi stage sequential classes and script for calling the above main loop function

In [ ]:
reset_seeds(123)
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression, ElasticNet
from sklearn.ensemble import BaggingClassifier
import time 
from sklearn.base import clone
import pickle

num_class = 2

stage_1 = ['NYHA_2', 'NYHA_3', 'NYHA_4', 'Race_1', 'Race_2', 'Race_3', 'Race_4','Race_5', 'Age',  'ACEI_or_ARB', 'CABG', 'CAD', 'DM','Gender', 'HTN',  'MI', 'PCI', 'Smoking']
stage_2 = ['LBBB', 'ECG_pre_QRSd' ]
stage_3 = ['SPECT_pre_LVEF', 'SPECT_pre_ESV','SPECT_pre_EDV', 'SPECT_pre_WTper', 'SPECT_pre_PSD', 'SPECT_pre_PBW', 'SPECT_pre_50scar', 'SPECT_pre_gMyoMass', 'SPECT_pre_WTsum',
            'SPECT_pre_StrokeVolume', 'SPECT_pre_PhaseKurt','SPECT_pre_Diastolic_PhasePeak', 'SPECT_pre_Diastolic_PhaseSkew','SPECT_pre_Diastolic_PBW', 'SPECT_pre_Diastolic_PSD', 'SPECT_pre_EDSI',
            'SPECT_pre_EDE', 'SPECT_pre_ESSI', 'SPECT_pre_ESE','SPECT_pre_Diastolic_PhaseKurt', 'SPECT_pre_PhasePeak', 'PECT_pre_SRscore',  'Concordance']


class Bootstraps:
    """
    n_models - # of models in ensemble
    model_itself - base model to use (RF, svm, enet, etc...) 
    sampling_fraction - fraction of samples for psuedo-bootstrap
    
    """
    def __init__(self, data_x, data_y, n_models, model_itself, sampling_fraction):
        self.data_x = data_x
        self.data_y = data_y
        self.n_models = n_models
        self.model_itself = model_itself
        self.sampling_fraction = sampling_fraction
        self.list_of_models = []

    def fit(self): 
        fitted_models = []
        for num in range(self.n_models):
            reset_seeds(123 + num)  
            model = clone(self.model_itself)
            np.random.seed(123 + num) 

            resample_index = np.random.choice(np.arange(len(self.data_y)), size = round(self.sampling_fraction * len(self.data_y)), replace = True).tolist()

            model.fit(self.data_x[resample_index], self.data_y[resample_index]);
            fitted_models.append(model)
            
            del model, resample_index
        
        self.list_of_models = fitted_models
        #return fitted_models
    
    def evaluate(self, test_data_x):
        probabilities = []
        for trained_model in self.list_of_models:
            prob_predictions = trained_model.predict_proba(test_data_x)
            probabilities.append(prob_predictions[:,1])
        
        probabilities = np.array(probabilities)
        avg_probabilities = probabilities.mean(axis = 0)
        avg_class = np.where(avg_probabilities >= 0.50, 1, 0)
        std = probabilities.std(axis = 0)
        
        class_counts = probabilities.round()
        num_response = np.sum(class_counts, axis = 0 )
        num_majority_class = np.where(num_response <= self.n_models/2, self.n_models - num_response, num_response)
        
        return avg_probabilities, avg_class, std, num_majority_class
    
    def return_importance(self):
        importance = [] 
        for trained_model in self.list_of_models:
            result = permutation_importance(trained_model,  self.data_x, self.data_y, n_repeats=10,
                                 random_state=123)
            importance.append(result.importances_mean)
            
        importance = np.array(importance).mean(axis = 0)
        return importance
    

class build_sequential_model():
    """
    std1_thresh - for deciding rules
    halfway_thresh - for deciding rules
    model_list - list of length 2 for the 2 stages which contains the ensemble models
    """
    def __init__(self, std1_thresh,halfway_thresh, model_list):
        
        self.std1_thresh = std1_thresh
        self.model_list = model_list
        self.halfway_thresh = halfway_thresh
        
    def evaluate(self, test_data_list):
        """
        aggregated_probabilities = final probabilities which stem from either the first or second model based off the rules
        keep = whether or not the sample should be predicted using the first model ( 1 = predict using first model, i.e., the sample did not need spect data)
        """
        print('test_data_list[0].shape',test_data_list[0].shape)
        aggregated_probabilities = np.zeros(shape = test_data_list[0].shape[0])
        keep = []
        standard_deviations = []
        for stage, (modell, data) in enumerate(zip(self.model_list, test_data_list)):
            print('stage', stage, 'date shape', data.shape)
            if stage == 0:
                avg_probabilities, _, std, _ = modell.evaluate(data)
                keep.append(np.where(np.logical_and(std <= self.std1_thresh, abs(avg_probabilities - 0.5) > self.halfway_thresh), 1, 0))
                aggregated_probabilities += np.where(keep[0] == 1, avg_probabilities, 0)
                standard_deviations.append(std)
                
            elif stage == 1:
                avg_probabilities, _, std, _ = modell.evaluate(data)
                aggregated_probabilities += np.where(keep[0] == 1, 0, avg_probabilities)
                standard_deviations.append(std)

        return aggregated_probabilities, standard_deviations, keep
    
    
stage_1 = ['NYHA_2', 'NYHA_3', 'NYHA_4', 'Race_1', 'Race_2', 'Race_3', 'Race_4','Race_5', 'Age',  'ACEI_or_ARB', 'CABG', 'CAD', 'DM','Gender', 'HTN',  'MI', 'PCI', 'Smoking']
stage_2 = ['LBBB', 'ECG_pre_QRSd' ]
stage_3 = ['SPECT_pre_LVEF', 'SPECT_pre_ESV','SPECT_pre_EDV', 'SPECT_pre_WTper', 'SPECT_pre_PSD', 'SPECT_pre_PBW', 'SPECT_pre_50scar', 'SPECT_pre_gMyoMass', 'SPECT_pre_WTsum',
            'SPECT_pre_StrokeVolume', 'SPECT_pre_PhaseKurt','SPECT_pre_Diastolic_PhasePeak', 'SPECT_pre_Diastolic_PhaseSkew','SPECT_pre_Diastolic_PBW', 'SPECT_pre_Diastolic_PSD', 'SPECT_pre_EDSI',
            'SPECT_pre_EDE', 'SPECT_pre_ESSI', 'SPECT_pre_ESE','SPECT_pre_Diastolic_PhaseKurt', 'SPECT_pre_PhasePeak', 'PECT_pre_SRscore',  'Concordance']




############################## Call experiment via this script ##############################

def score(dataframe,  save_path,fs_method, smotee, spatial_sign, what_model  ):
    """
   Main function for running experiement 
    fs_method - feature selection method 
    smotee - if to use SMOTE
    spatial_sign - if to use spatial sign 
    what_model - what base model to use (RF, svm, etc...)
    """

    clinical_data = dataframe.drop(columns = ['resp', 'Race', 'NYHA', 'Hospitalization'])
    
    
    # making data for different stages 
    X2 = np.array(clinical_data.filter(stage_1 + stage_2))
    X3 = np.array(clinical_data.filter(stage_1 + stage_2 + stage_3))
 
    # define global features 
    global stage1_features
    stage1_features = clinical_data.filter(stage_1 + stage_2).columns
 
    global stage2_features 
    stage2_features = clinical_data.filter(stage_1 + stage_2 + stage_3).columns
    
    # foprmat data
    Y = np.where(np.array(response)  == 'yes', 1, 0)    
   
    # evaluate ML
    stage0_wt = [1] # weight parameter for scaling function for weighted auc 
    stage1_wt = [1]
    for weights in product(stage0_wt, stage1_wt):

        combo_output = []
        scores = np.zeros((10,5))
        idx = 0
        ind_scores = [] 
        sequential_mods = []
        stand_dev = []
        boot_model_hypers = []
        stage_transitions = []
        val_stand_dev = []
        val_stage_transitions = []
        scle_weights = []
        ft_import = []
        auc_curves = []
        ft_used = []
        crnt_fold = 0 
        for train, test in StratifiedKFold(n_splits=10, shuffle = True, random_state = 123).split(X2, Y):
            print(len(train))
            print('Fold', idx)
            scores[idx], ind_performance, mods, stds, boot_hp, stage_trans, std_val, stage_trans_val, scaling_weight, feature_importances, auc_curv, feature_names = train_evaluate( X2[train], X3[train], Y[train],  X2[test], X3[test], Y[test], train, test, fs_type = fs_method, smote_bol = smotee , spatial = spatial_sign, models = what_model, fold =  crnt_fold)
            idx += 1
            ind_scores.append(ind_performance)
            sequential_mods.append(mods)
            stand_dev.append(stds)
            boot_model_hypers.append(boot_hp)
            stage_transitions.append(stage_trans)
            val_stand_dev.append(std_val)
            val_stage_transitions.append(stage_trans_val)
            scle_weights.append(scaling_weight)
            ft_import.append(feature_importances)
            auc_curves.append(auc_curv)
            ft_used.append(feature_names)
            crnt_fold += 1
        scores = pd.DataFrame(scores)
        #scores.to_csv(save_path , mode='a', header=not os.path.exists(save_path ), index = False)
    
        data_run = [scores, ind_scores, sequential_mods, stand_dev, boot_model_hypers, stage_transitions, val_stand_dev, val_stage_transitions, scle_weights, ft_import, auc_curves, ft_used]
   
        with open(what_model + '_' +  str(1) + '_' +  str(1) + 'final_please', "wb") as fp:   #Pickling
            pickle.dump(data_run, fp)
   
   

with smote and spatial sign (repeat)

In [ ]:
score(df, save_path = '/root/uncertainty_multi_stage/enet_weights_new_weight_func.csv', fs_method = 'lr', smotee = True, spatial_sign = True, what_model = 'enet')

In [ ]:

# =======================================
# Processing pickle files for displaying raw results 
# =======================================


with open('/root/uncertainty_multi_stage/enet_1_1final_please', "rb") as fp:   # Unpickling
    b = pickle.load(fp)



score = b[0].rename({0:'AUC',1:'Accuracy',2:'Kappa',3:'Sensitivity',4:'specificity'}, axis = 1)
print('Avg score',score.mean(axis = 0))
print('std score',score.std(axis = 0))

print('Raw Scores', score)

print('Model scores')
for model in range(2):
    model_scores = []
    for cv in range(10):
        ind = b[1][cv][model] #.rename({0:'AUC',1:'Accuracy',2:'Kappa',3:'Sensitivity',4:'specificity'}, axis = 1)
        model_scores.append(ind)
        print('CV ', cv,' Model: ', model, 'Score: ', ind)
    print('             Model ', model, ' Avg ', np.array(model_scores).mean(axis = 0 ))
    print('             Model ', model, ' std ', np.array(model_scores).std(axis = 0 ))

print('std thresholds')
for cv in range(10):
    print('CV ', cv, ' std1_thresh', (vars(b[2][cv]).get('std1_thresh')))
    print('CV ', cv, ' midway thresh', (vars(b[2][cv]).get('halfway_thresh')))
    
print('std')
for model in range(2):
    std_overall = []
    for cv in range(10):
        ind = b[3][cv][model] #.rename({0:'AUC',1:'Accuracy',2:'Kappa',3:'Sensitivity',4:'specificity'}, axis = 1)
        std_overall.append(ind)
        print('CV ', cv,' Model: ', model, 'Score: ', ind)
    print('             Model ', model, ' Avg ', ((np.array(std_overall[:8]).sum() + np.array(std_overall[8:]).sum())/(np.array(std_overall[:8]).size + np.array(std_overall[:8]).size)))
    
print('hyperparams')
for model in range(2):
    boot_MODELS = []
    for cv in range(10):
        ind = b[4][cv][model] #.rename({0:'AUC',1:'Accuracy',2:'Kappa',3:'Sensitivity',4:'specificity'}, axis = 1)
        boot_MODELS.append(ind)
        print('CV ', cv,' Model: ', model, 'Score: ', ind)
    print('             Model ', model, ' Avg ', np.array(boot_MODELS).mean(axis = 0 ))
    
print('stg movements')
for model in range(1):
    std_overall = []
    for cv in range(10):
        ind = b[5][cv][model] #.rename({0:'AUC',1:'Accuracy',2:'Kappa',3:'Sensitivity',4:'specificity'}, axis = 1)
        std_overall.append(ind)
        print('CV ', cv,' Model: ', model, 'Score: ', ind)
    print('             Model ', model, ' Avg ', (np.array(std_overall[:8]).sum() + np.array(std_overall[8:]).sum()/(np.array(std_overall[:8]).size + np.array(std_overall[:8]).size)))
    
    
print('scaling weight function')
for cv in range(10):
    ind = b[8][cv] #.rename({0:'AUC',1:'Accuracy',2:'Kappa',3:'Sensitivity',4:'specificity'}, axis = 1)
    print('CV ', cv, ' Scaling Weight: ', ind)

    
        
        
        

In [ ]:
print('stg movements')

std_overall = []
SUMS = []   
for cv in range(10):
    ind = b[5][cv][0] #.rename({0:'AUC',1:'Accuracy',2:'Kappa',3:'Sensitivity',4:'specificity'}, axis = 1)
    std_overall.append(ind)
    SUMS.append(sum(ind)/len(ind))
    print('CV ', cv,' Model: ', model, 'Score: ', ind)
print('percent of patients requiring spect', 100 - np.mean(SUMS) * 100)
print(np.std(SUMS) * 100)
print('25th percentile', np.percentile(np.array(SUMS), 25) * 100)
print('50th percentile', np.percentile(np.array(SUMS), 50)* 100)
print('75th percentile', np.percentile(np.array(SUMS), 75)* 100)

In [ ]:
list_of_roc_curve = []
for i in range(10):
    list_of_roc_curve.append(b[10][i][2]) # vary last slice from 0, 1, 2 for ensemble 1, ensemble 2, and multistage 
    
from sklearn.metrics import auc

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots(figsize=(8, 8))
for i in range(10):
    viz = list_of_roc_curve[i]
    viz.plot(ax = ax, alpha = 0.3)
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)


ax.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title=f"Mean ROC curve with variability\n(Multi-stage)",
)
ax.axis("square")
ax.legend(loc="lower right")
plt.show()